In [1411]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [1441]:
table = pd.read_table('../drug-discovery/1663769555_8559356_train.txt', names=['binding','sequence'])

In [1413]:
def collect_with_target_value(data_frame, column, target_value):
    collected_values = data_frame.loc[data_frame[column] == target_value]
    return collected_values

In [1414]:
active_df = collect_with_target_value(table, 'binding', 1).reset_index()
not_active_dfs = collect_with_target_value(table, 'binding',0).reset_index()

In [1415]:
not_active_dfs.shape

(722, 3)

In [1416]:
not_active_dfs = pd.DataFrame.sample(not_active_dfs,frac=.30, random_state= np.random.RandomState())
not_active_dfs.binding.shape

(217,)

In [1417]:
final_data_set = pd.concat([active_df,not_active_dfs])
final_data_set = final_data_set.drop(['index'],axis=1).sample(frac=1).reset_index()
final_data_set = final_data_set.drop(['index'],axis=1)

In [1418]:
import imblearn
from imblearn.over_sampling import RandomOverSampler
sampler = RandomOverSampler()
x_sampled, y_sampled = sampler.fit_resample(np.array(table.sequence).reshape(-1,1),table.binding)

In [1419]:
final_data_set = pd.DataFrame(x_sampled, columns  =["sequence"])
final_data_set['binding'] = y_sampled
final_data_set.binding.value_counts()

0    722
1    722
Name: binding, dtype: int64

In [1420]:
from sklearn.model_selection import  train_test_split

training_split, test_split, train_result, test_result = train_test_split(final_data_set.sequence, final_data_set.binding,stratify=final_data_set.binding, train_size=.70, random_state=0)

In [1421]:
test_result.shape

(434,)

In [1422]:
from sklearn.feature_extraction.text import CountVectorizer
def return_binary_vectorized_form(data_frame):
    count_vectorizer = CountVectorizer(binary=True)
    fitted_and_transformed = count_vectorizer.fit_transform(data_frame).toarray()
    return count_vectorizer,  fitted_and_transformed
vectorizer, training_split = return_binary_vectorized_form(training_split)
test_split = vectorizer.transform(test_split).toarray()

In [1423]:
test_split

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [1424]:
train_result = np.array(train_result)
test_result = np.array(test_result)

In [1425]:
# training_sequence= []
# test_sequence = []
# for row in training_split:
#     training_sequence.append(str(row))
# for row in test_split:
#     test_sequence.append(str(row))


In [1426]:
training_split

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [1427]:
test_split
# training_sequence = np.array(training_sequence).reshape(1010)
# # test_sequence = np.array(test_sequence)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [1428]:
# training_sequence.s

In [1429]:
vocab_size = 40000
embedding_dim = 16
max_length = 1000
trunc_type = 'post'
oov_tok = '<OOV>'
padding_type = 'post'

In [1430]:
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
# tokenizer.fit_on_texts(training_sequence)
# word_index = tokenizer.word_index

In [1431]:
# sequences = tokenizer.texts_to_sequences(training_sequence)
# padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)
# test_sequence = tokenizer.texts_to_sequences(test_sequence)
# testing_padded = pad_sequences(test_sequence, maxlen=max_length)
# padded

In [1432]:
training_split.shape[-1]

82172

In [1532]:
model = tf.keras.Sequential([
    tf.keras.Input(shape=(training_split.shape[-1],)),
    tf.keras.layers.Dropout(.8),
    tf.keras.layers.Dense(26,kernel_regularizer=tf.keras.regularizers.L2(0.1), activation='relu'),
    tf.keras.layers.Dropout(.8),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [1533]:
METRICS = [
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'),
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
      tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), optimizer='adam', metrics=METRICS)

In [1534]:
num_epochs = 2
history = model.fit(training_split, train_result, epochs=num_epochs, validation_data=(test_split, test_result))

Epoch 1/2


/Users/bhabaranjanpanigrahi/miniforge3/envs/mlp/lib/python3.8/site-packages/keras/backend.py:5673: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
2022-10-04 22:39:41.958461: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


32/32 [==============================] - ETA: 0s - loss: 1.4680 - tp: 339.0000 - fp: 137.0000 - tn: 368.0000 - fn: 166.0000 - accuracy: 0.7000 - precision: 0.7122 - recall: 0.6713 - auc: 0.7813 - prc: 0.8019

2022-10-04 22:39:48.970618: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


32/32 [==============================] - 10s 205ms/step - loss: 1.4680 - tp: 339.0000 - fp: 137.0000 - tn: 368.0000 - fn: 166.0000 - accuracy: 0.7000 - precision: 0.7122 - recall: 0.6713 - auc: 0.7813 - prc: 0.8019 - val_loss: 1.0030 - val_tp: 205.0000 - val_fp: 4.0000 - val_tn: 213.0000 - val_fn: 12.0000 - val_accuracy: 0.9631 - val_precision: 0.9809 - val_recall: 0.9447 - val_auc: 0.9915 - val_prc: 0.9899
Epoch 2/2
32/32 [==============================] - 2s 74ms/step - loss: 1.0115 - tp: 446.0000 - fp: 58.0000 - tn: 447.0000 - fn: 59.0000 - accuracy: 0.8842 - precision: 0.8849 - recall: 0.8832 - auc: 0.9571 - prc: 0.9613 - val_loss: 1.0416 - val_tp: 217.0000 - val_fp: 4.0000 - val_tn: 213.0000 - val_fn: 0.0000e+00 - val_accuracy: 0.9908 - val_precision: 0.9819 - val_recall: 1.0000 - val_auc: 0.9969 - val_prc: 0.9965


In [1535]:
table_test = pd.read_table('../drug-discovery/1663769555_8653905_test.txt', names=['sequence'])

In [1536]:
# unlabeled_test_sequence = []
# for row in table_test.sequence:
#     unlabeled_test_sequence.append(str(row))

In [1537]:
table_test

,sequence
0,259 521 546 565 671 769 858 876 1056 1252 1392...
1,96 158 195 436 518 541 588 642 804 843 978 115...
2,13 169 354 587 803 1241 1472 1586 1605 1638 16...
3,10 91 184 191 546 825 978 1077 1155 1220 1297 ...
4,37 85 119 191 349 390 437 458 671 707 1454 157...
...,...
345,43 150 273 363 797 889 1014 1016 1077 1176 135...
346,34 118 132 150 162 259 347 470 477 479 613 651...
347,84 250 263 616 1220 1289 1314 1384 1440 1442 1...
348,121 136 165 207 232 309 375 412 453 460 521 72...


In [1538]:
unlabeled_test_sequence = vectorizer.transform(table_test.sequence).toarray()

In [1539]:
unlabeled_test_sequence

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [1540]:
# unlabeled_test_sequence = tokenizer.texts_to_sequences(unlabeled_test_sequence)
# padded_unlabeled_test_sequence = pad_sequences(unlabeled_test_sequence, maxlen=max_length)

In [1541]:
y = model.predict(unlabeled_test_sequence)

 2/11 [====>.........................] - ETA: 0s

2022-10-04 22:40:10.776889: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


11/11 [==============================] - 1s 34ms/step


In [1542]:
y

array([[0.64994174],
       [0.20488484],
       [0.29602903],
       [0.28682467],
       [0.37527844],
       [0.2976266 ],
       [0.28580004],
       [0.36950392],
       [0.3227047 ],
       [0.483375  ],
       [0.27841666],
       [0.33279234],
       [0.16148861],
       [0.3969377 ],
       [0.26585257],
       [0.36078355],
       [0.14083426],
       [0.24140169],
       [0.24891168],
       [0.2861667 ],
       [0.1586035 ],
       [0.34005073],
       [0.1131246 ],
       [0.3177263 ],
       [0.4752832 ],
       [0.6789998 ],
       [0.3330198 ],
       [0.2480396 ],
       [0.32258627],
       [0.27950126],
       [0.27997562],
       [0.2819417 ],
       [0.2581854 ],
       [0.12148857],
       [0.3038008 ],
       [0.27870005],
       [0.32511216],
       [0.35933593],
       [0.45274305],
       [0.25146   ],
       [0.27368754],
       [0.2990703 ],
       [0.29429534],
       [0.17278609],
       [0.16926172],
       [0.24480575],
       [0.26086706],
       [0.287

In [1543]:

# for i in range(3):
z = [1 if i>=.5 else 0 for i in y ]

In [1544]:
# from sklearn.metrics import f1_score
# print(f1_score(z,test_result))

In [1545]:
z[61]

1

In [1546]:
len(z)

350

In [1547]:
import csv
outfile = open('./results.csv','w')
out = csv.writer(outfile)
out.writerows(map(lambda x: [x], z))
outfile.close()

In [1548]:
len(z)

350

In [1500]:
len(tf.config.list_physical_devices('GPU'))>0

True

In [1501]:
import platform
platform.platform()

'macOS-12.6-arm64-arm-64bit'